<a id='top'></a>
# A Quantum Approach for Fault Tree Analysis #


## Table of Contents
1. [Introduction](#introduction)
2. [Fault Tree Analysis Problem](#prob_description)
3. [Classical Solution using OR-Tools](#classical_solution)
4. [Conversion of Fault Tree to the Quantum Regime](#quantum_solution)
5. [Setting up the Problem using the QC Ware API](#problem)
    <br>5.a. [Defining the Q Matrix](#Q_matrix)
    <br>5.b. [Solving the Problem](#solve)


## 1. Introduction <a name="introduction"></a>

This demo explores the use of quantum computing (QC) for the analysis of fault trees for engineering design and testing. 

In several industrial domains including energy, oil & gas, pharmaceuticals, electronics, and aerospace, significant computing resources are applied to system design, testing, and verification, especially related to safety compliance and regulations. Of particular interest is fault tree analysis (FTA)–determining the most probable sets of failure events in a complex system that lead to a top-level failure.

Fault tree analysis (FTA) is a top-down, deductive failure analysis in which an undesired state of a system is analyzed using Boolean logic to combine a series of lower-level events. We aim to understand how the system can fail, to identify the best ways to reduce the failure probability, or to determine the probability of a particular system failure.

For instance, NASA uses a form of FTA in order to determine the likelihood of catastrophic failure and to assess quantitative risks. They might, for instance, take the hypothetical case of a launch failure and work backwards to determine the most likely cause given their current protocols, hardware, etc. By understanding the most likely pathway 
that would lead to launch failure, NASA not only gets a better sense of how safe their launch is, but also gets the opportunity to improve their protocols based on the fault tree they develop.

***
## 2. Fault Tree Analysis Problem <a name="prob_description"></a>

The fault-tree that we aim to study in this notebook looks like this:
![](https://files.slack.com/files-pri/T24940PQV-FFR7K200K/fault_tree.png?pub_secret=f370b437dc)


Here we have a top event ($p_0$) at the top, and a number of input events ($e_0, e_1,e_2, e_3$) at the bottom that correspond to a failure or error in a system component or element. The input events can take the value 0 or 1: either there is a failure on that particular component (1) or it is working properly (0). Failure of the top event would correspond to system failure. The input events are connected to each other as inputs of logic gates ($G_1,G_2,G_3,G_4$). Boolean logic in the form of a series of logic gates is used to determine what value the top event takes. These logic gates form the fault-tree that we will use in this demo. The output of each gate ($z_0, z_1, z_2$) will act as input of other gates.

The aim of this analysis is to find the smallest or most probable set of input events that by failing would lead to a top event failure. In general, there often are several sets of this size and thus we may find ourselves being intersted in more than one set of input events. What we are looking for is to determine how probable it is for the system to fail, or what conditions needs to be met in order for the most probable system failure to occur. This is done for safety reasons, to guarantee that the probability of the system failing is lower than the imposed treshold.

Below we show the minimum cut set for the fault-tree we are studying, it corresponds to the configuration $e_0 = z_0 = z_1 = z_2 =1$ and $e_1=e_2=e_3=0$.

![](https://files.slack.com/files-pri/T24940PQV-FFSP2V18E/fault_tree_cut_set.png?pub_secret=352f426394)

Only by knowing the values of the input events ($e_i$) can the values of the other variables be derived following the logic of the fault tree. It is easy to see that this configuration leads to a top event failure $p_0=1$.



***
## 3. Classical solution using ```OR-Tools```<a name="classical_solution"></a>
```OR-Tools``` is an open source software suite for optimization, available from Google. It is possible to configure ```OR-Tools``` to use commercial solvers like ```CPLEX``` or ```Gurobi```, or open-source solvers like ```SCIP``` or ```GLPK```, but this involves building ```OR-Tools``` from source, and we will not discuss this here. Instead we will focus on using Google's ```GLOP``` solver which is available upon a standard installation. More information on ```OR-Tools``` can be found at the <a href="https://developers.google.com/optimization/introduction/overview" style="text-decoration: none;">OR-Tools homepage</a>. The user guide can be found <a href="https://jupyter-notebook.readthedocs.io/en/stable/notebook.html" style="text-decoration: none;">here</a>, which contains extensive documentation and lots of examples.

Our fault tree diagnosis problem can be expressed as a binary optimization problem. We will assign a binary variable $e_i \in \{0,1\}$ for every input event. Logic gate outputs will also be binary variables $z_j \in \{0,1\}$. Then our fault tree diagnosis problem can be expressed by
$$
\begin{equation}
\begin{split}
\text{minimize} \;\; & \sum_{i} e_i  \\
\text{subject for each OR gate to} \;\; & y_1 \ge x_1+ x_2 \;\; y_1 \le x_1 \;\; y_1 \le x_2 \\
\text{subject for each AND gate to} \;\; & y_1 \le x_1+ x_2-1 \;\; y_1 \ge x_1 \;\; y_1 \ge x_2
\end{split}
\end{equation}
$$

Here $y_1$ corresponds to the output of the gate and $x_1$, $x_2$ to the inputs.

We first import the solver library from ```ortools```.

In [1]:
from qcware import forge
# this line is for internal tracking; it is not necessary for use!
forge.config.set_environment_source_file('fault_tree.ipynb')

#NBVAL_IGNORE_OUTPUT
!pip install ortools
from ortools.linear_solver import pywraplp

     |▉                               | 378 kB 4.7 MB/s eta 0:00:04

     |█████▌                          | 2.5 MB 4.7 MB/s eta 0:00:03

     |███████████▎                    | 5.1 MB 4.7 MB/s eta 0:00:02

     |████████████████▋               | 7.5 MB 4.7 MB/s eta 0:00:02

     |██████████████████████▋         | 10.3 MB 4.7 MB/s eta 0:00:01

...

     |████████████████████████████████| 14.5 MB 4.7 MB/s eta 0:00:01

     |████████████████████████████████| 14.5 MB 4.7 MB/s            


     |██████████████████▎             | 604 kB 44.6 MB/s eta 0:00:01

     |████████████████████████████████| 1.1 MB 44.6 MB/s            


     |████████████████████████████████| 126 kB 50.8 MB/s            


Next we instantiate the classical solver, and obtain the ```solver``` object.

In [2]:
# Instantiate a mixed-integer solver, naming it fault-tree-diagnosis
solver = pywraplp.Solver('fault-tree-diagnosis', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

First, we define the binary problem variables.

In [3]:
# Define integer binary variables.
e0 = solver.IntVar(0, 1, 'e0')
e1 = solver.IntVar(0, 1, 'e1')
e2 = solver.IntVar(0, 1, 'e2')
e3 = solver.IntVar(0, 1, 'e3')
z0 = solver.IntVar(0, 1, 'z0')
z1 = solver.IntVar(0, 1, 'z1')
z2 = solver.IntVar(0, 1, 'z2')

Next, we set the constraints of the problem. To set the constraints, we will have to go over each gate.

In [4]:
## Constraint Gate 1: 0 <= e0 + e3 - z0 ; z0 - e0 >= 0 ; z0 - e3 >= 0
constraint1 = solver.Constraint(0, solver.infinity())
constraint1.SetCoefficient(e0, 1)
constraint1.SetCoefficient(e3, 1)
constraint1.SetCoefficient(z0, -1)

constraint1 = solver.Constraint(0, solver.infinity())
constraint1.SetCoefficient(e0, -1)
constraint1.SetCoefficient(z0, 1)

constraint1 = solver.Constraint(0, solver.infinity())
constraint1.SetCoefficient(e3, -1)
constraint1.SetCoefficient(z0, 1)

# Constraint Gate 2: 0 <= e2 + z0 - z1 ; z1 - e2 >= 0 ; z1 - z0 >= 0
constraint1 = solver.Constraint(0, solver.infinity())
constraint1.SetCoefficient(e2, 1)
constraint1.SetCoefficient(z0, 1)
constraint1.SetCoefficient(z1, -1)

constraint1 = solver.Constraint(0, solver.infinity())
constraint1.SetCoefficient(e2, -1)
constraint1.SetCoefficient(z1, 1)

constraint1 = solver.Constraint(0, solver.infinity())
constraint1.SetCoefficient(z0, -1)
constraint1.SetCoefficient(z1, 1)

# Constraint Gate 3: 0 <= e0 + e1 - z2 ; z2 - e0 >= 0 ; z2 - e1 >= 0
constraint1 = solver.Constraint(0, solver.infinity())
constraint1.SetCoefficient(e0, 1)
constraint1.SetCoefficient(e1, 1)
constraint1.SetCoefficient(z2, -1)

constraint1 = solver.Constraint(0, solver.infinity())
constraint1.SetCoefficient(e1, -1)
constraint1.SetCoefficient(z2, 1)

constraint1 = solver.Constraint(0, solver.infinity())
constraint1.SetCoefficient(e0, -1)
constraint1.SetCoefficient(z2, 1)

# Constraint Gate 4: -2 <= -z1 - z2 ; z1 >= 1 ; z2 >= 1
constraint1 = solver.Constraint(-2, solver.infinity())
constraint1.SetCoefficient(z1, -1)
constraint1.SetCoefficient(z2, -1)

constraint1 = solver.Constraint(1, solver.infinity())
constraint1.SetCoefficient(z1, 1)

constraint1 = solver.Constraint(1, solver.infinity())
constraint1.SetCoefficient(z2, 1)


The objective function for the problem is then set.

In [5]:
# Minimize e0+e1+e2+e3
objective = solver.Objective()
objective.SetCoefficient(e0, 1)
objective.SetCoefficient(e1, 1)
objective.SetCoefficient(e2, 1)
objective.SetCoefficient(e3, 1)
objective.SetMinimization()

We finally solve the problem and then verify that an optimal solution has been found.

In [6]:
result_status = solver.Solve()

assert result_status == pywraplp.Solver.OPTIMAL

Print the solution.

In [7]:
print("\n")
print("Faulty event(s) that lead to a top event failure:")
for item in ['e0', 'e1', 'e2', 'e3']:
    var = solver.LookupVariable(item)
    if var.solution_value() == 1:
        print(item)



Faulty event(s) that lead to a top event failure:
e0


***
## 4. Conversion of Fault Tree to the Quantum Regime<a name="quantum_solution"></a>

This section provides a detailed explanation on how to transform fault tree logic first to PUBO (polynomial unconstrained binary optimization) and then to QUBO (quadratic unconstrained binary optimization) notation that can be used as input to the quantum computer. 

Our aim is to show that a fault tree can be represented by a Pseudo-boolean function of the form 

$$
H_{PUBO}(e_n,z_n,p_0)=H_{consist}(e_n,z_n,p_0)+H_{numfaults}(e_n,z_n,p_0)
$$ 

where $e_n,z_n,p_0 \in \{0,1\}$ are binary variable vectors representing the base events, outputs of gates, and top failure, respectively. A cut set is an assignment of values  $e_n,z_n \to v_n \in \{0,1\}, p_0 \to 1$ that satisfies $H_{consist}(v_n,1)=0$ and $H_{PUBO}(v_n,1)=N_{numfaults}$ where $N_{numfaults}$ is the number of base events that satisfy $e_i=1$. The minimal cut set is a cut set $v_n$ with a minimum number of ones in $e_n$ e.g. for a length 4 vector $e_n=(e_0, e_1, e_2, e_3)$ a cut set would correspond to $(1, 1, e_2, e_3)$ and a sample minimal cut set would be $(1, 0, 0, 0)$ as long as $H_{consist}(v_n,1)=0$ for both solutions. Invalid solutions $w_n$ would result in a penalty of the form $H_{consist}(w_n,1)>0$. Thus, the minimum value of the $H_{PUBO}$ function would correspond to the shortest minimal cut set solution, higher value solutions would be minimal cut sets and super cut sets, and the highest values are given by invalid solutions. Our aim is to generate the $H_{PUBO}$ objective function that verifies mathematically the following two claims: an assignment $V$ such that the cost function $H_{PUBO}(V)$ takes its absolute minimum $H_{PUBO/min}$ corresponds to the shortest minimal cut set of the tree; we cannot claim the converse, that to every cut set there is an assignment $V$ such that $H_{PUBO}(V)=H_{PUBO/min}$. However, the quantum computer will generate solutions at and near the minimum in a probabilistic manner, finding shortest minimal cut sets, minimal cut sets, and super cut sets, and we will use postprocessing to validate solutions and to generate the final minimal cut set list.

We can construct a PUBO polynomial whose ground states encode the solution to the shortest minimal cut set for a given fault tree by adding the following two terms to construct an objective function, 

$$
H_{PUBO}=H_{consist}+H_{numfaults} .
$$

We describe next each of these terms, $H_{numfaults}$ is the addition of all the event of the fault tree. It takes the value of the number of faulty events in the solution.

$$
H_{numfaults}(\{ e_i \})= \sum e_i 
$$

$H_{consist}$ is the summation of the $H_{gates}$ objective functions for all the gates present in the fault tree.

$$
H_{consist}=\sum_{j \in gates} H^{(j)}_{gates}
$$

The $H_{gates}$ objective function represents the consistency of the logic of the gate i.e. it assigns a penalty to the solutions that don't fulfill the logic of that gate and takes minimum value when the logic is satisfied by the input and output variables of the gate. It reads,

$$
H^{(j)}_{gates}(\{ \textbf{x} \},z_j,e_j)=\lambda^{(j)}_{consist} XOR(gate_j(\{ \textbf{x}_j \}_j), z_{j-1}) 
$$

Here $z_j$ is the output, $\textbf{x}_j$ the set of input variables for the $j$-th gate and $gate_j(\{ \textbf{x}_j \}_j)$ a Boolean function denoting the expected nominal output from the $j$-th component. Input variables can be either outputs of other gates $z_k$ or events $e_k$. The $gate_j()$ function varies depending on the gate type, $XOR()$ it is just a particular case of this function, below you can find the expressions to be used for the nominal output of different gates with one or two inputs.

$$
AND(x_1,x_2)=x_1x_2
$$
$$
OR(x_1,x_2)=x_1+x_2-x_1x_2
$$ 

The $\lambda^{(i)}_{consist}$ prefactor shows the relative importance of $H_{consist}$ with respect to $H_{numfaults}$. In this case it is more important that the solution satisfies the logic of the circuit than the number of faulty events is minimized, as this would lead to a random solution that is independent of the logic of the tree. The value of $\lambda^{(i)}_{consist}$ is empirically set by the user to give the best performance for the problem class. For more information in the $\lambda^{(i)}_{consist}$ parameter see below.

Functions with higher than quadratic terms need to be reduced to that form. We will have to add ancilla variables of the form $q_{an}=q_i q_j$ to obtain the quadratic $H_{QUBO}$ objective function that can be used as input in the computer. It is not enough to make the substitution in the $H_{PUBO}$ expression to obtain $H_{QUBO}$, since the ancillas are new variables whose values are independent of the original variables, we need to penalize the incorrect variable assignments. We add a penalty term for each ancilla of the form
$$
E_{penalty}=\delta (3q_{an}+q_i q_j-2q_i q_{an}-2 q_j q_{an} )
$$
Where $\delta$ is a positive number representing the chosen penalty. 

Using these rules we can obtain the QUBO formulation of our sample fault-tree. As an exercise the reader could obtain this result and compare with the QUBO formulation obtained by the code below.

***
## 5. Setting up the Problem using the QC Ware Client API <a name="problem"></a>


Next, we explain the formulation that we use to input the logic of the fault-tree into our QC code. Below we show the input corresponding to our sample fault tree,

$$
input: e0
$$
$$
input: e1
$$
$$
input: e2
$$
$$
input: e3
$$
$$
output: p0
$$
$$
G1: z0 == e0 ~ || ~ e3
$$
$$
G2: z1 == e2 ~ || ~ z0
$$
$$
G3: z2 == e0 ~ || ~ e1
$$
$$
G4: p0 == z1 ~ \&\& ~ z2
$$

This notation shows the inputs, outputs, and gates ($G$) of the fault tree. The inputs are the 3 base events $e_0,e_1,e_2$. The output is the top event that we label $p_0$, it will take the value $p_0=1$ when we are searching for system failures. In the following lines we describe the logic of each gate in the tree described as $G_n$. Here $z_i$ represents the output of the gate $G_{i+1}$. The $\&\&$ symbol corresponds to AND gates and $||$ to OR gates. For more general fault-trees the number of inputs, gates, and outputs will increase which will lead to a more complex logic.

### 5.a. Defining the Q Matrix <a name="Q_matrix"></a>

For the purpose of this notebook we will now input the structure of the fault by hand to generate a Q matrix that fits the QUBO formulation that our qcware optimization module can solve. We will do this by following the steps specified in the previous "Mapping of the fault-tree analysis problem to QUBO" section. For a more detailed description of the Q matrix, please see the Getting Started Guide. 

We will use the Python package `qubovert` that we developed to aid in the construction of QUBOs for binary optimization problems. We will install and import it here, along with the QC Ware module and other neccesary models.

In [8]:
%%capture
!pip install qubovert

import qubovert as qv
import pprint

Next, we need to specify the penalty value that we assign to $\lambda$. In general it is very important to fine-tune this parameter! For this example, we will just set it to 4.

In [9]:
lam = 4

We use `qubovert` to create a Polynomial Constrained Boolean Optimization (PCBO) object. We want to minimize $H_{numfaults}$; we also encode this into the PCBO.

In [10]:
errors = 'e0', 'e1', 'e2', 'e3'

H = qv.PCBO()
for e in errors:
    H[(e,)] = 1
    
pprint.pprint(H)

{('e0',): 1, ('e1',): 1, ('e2',): 1, ('e3',): 1}


Next we enforce the constraints that create the `z`s (reproduced here for reference):
$$
G1: z0 == e0 ~ || ~ e3
$$
$$
G2: z1 == e2 ~ || ~ z0
$$
$$
G3: z2 == e0 ~ || ~ e1
$$
We will enforce these constraints with the penalty factor $\lambda$.

In [11]:
H.add_constraint_eq_OR(  # enforce that z0 = e0 OR e3
    'z0', 'e0', 'e3', lam=lam
).add_constraint_eq_OR(  # enforce that z1 = e2 OR z0
    'z1', 'e2', 'z0', lam=lam
).add_constraint_eq_OR(  # enforce that z2 = e0 OR e1
    'z2', 'e0', 'e1', lam=lam
)
pprint.pprint(H)

{('e0',): 9,
 ('e0', 'z0'): -8,
 ('e1',): 5,
 ('e1', 'e0'): 4,
 ('e2',): 5,
 ('e2', 'z0'): 4,
 ('e2', 'z1'): -8,
 ('e3',): 5,
 ('e3', 'e0'): 4,
 ('e3', 'z0'): -8,
 ('z0',): 8,
 ('z1',): 4,
 ('z1', 'z0'): -8,
 ('z2',): 4,
 ('z2', 'e0'): -8,
 ('z2', 'e1'): -8}


Finally we want to enforce the constraint that `p0 = 1` (reproduced here for reference)

$$
G4: p0 == z1 ~ \&\& ~ z2
$$

We want to make $z1 ~ \& \& ~ z2$ energetically favorable. This can be done without introducing the `p0` variable in the following way.

In [12]:
H.add_constraint_AND('z1', 'z2', lam=lam)
pprint.pprint(H)

{(): 4,
 ('e0',): 9,
 ('e0', 'z0'): -8,
 ('e1',): 5,
 ('e1', 'e0'): 4,
 ('e2',): 5,
 ('e2', 'z0'): 4,
 ('e2', 'z1'): -8,
 ('e3',): 5,
 ('e3', 'e0'): 4,
 ('e3', 'z0'): -8,
 ('z0',): 8,
 ('z1',): 4,
 ('z1', 'z0'): -8,
 ('z2',): 4,
 ('z2', 'e0'): -8,
 ('z2', 'e1'): -8,
 ('z2', 'z1'): -4}


Thus we have encoded our problem! Let's look at the final binary optimization problem.

In particular, we are interested in the degree of the model.

In [13]:
print(H.degree)

2


Notice that `H` is already degree 2 and is thus a QUBO! However, we will still perform the following steps since they work even when the degree of the HOBO is greater than 2.

In [14]:
qubo = H.to_qubo()
pprint.pprint(qubo)

{(): 4,
 (0,): 9,
 (0, 1): 4,
 (0, 3): 4,
 (0, 4): -8,
 (0, 6): -8,
 (1,): 5,
 (1, 6): -8,
 (2,): 5,
 (2, 4): 4,
 (2, 5): -8,
 (3,): 5,
 (3, 4): -8,
 (4,): 8,
 (4, 5): -8,
 (5,): 4,
 (5, 6): -4,
 (6,): 4}


Notice that the QUBO is labeled with integers, whereas the HOBO was labeled with strings. We can convert between the labels with the `H.convert_solution` method, discussed below.

### 5.b. Solving the Problem <a name="solve"></a>

We're now ready to solve the problem using QC Ware Platform's optimization function. We just need to input the BinaryProblem class, the solver that we aim to use, and our token. To ensure our formulation is correct, we will use a bruteforce solver, but you can change the `solver` keyword to any backend you would like.

The `qcware` `optimize_binary` function accepts the `BinaryProblem` class that is generated out of `qubovert`'s QUBOs. 

Notice also that we convert between the integer labeled QUBO and the string labeled HOBO with `H.convert_solution`. The `H.is_solution_valid` method returns whether or not all the constraints are satisfied.

In [15]:
import qcware.types
poly = qcware.types.optimization.PolynomialObjective(
    polynomial=qubo.Q,
    num_variables=qubo.num_binary_variables,
    domain='boolean'
)
problem = qcware.types.optimization.BinaryProblem(objective=poly)

result = forge.optimization.optimize_binary(instance=problem, backend='qcware/cpu')
sol_qubo = result.lowest_value_bitstring
solution = H.convert_solution(sol_qubo)

print('QUBO solution:', sol_qubo)
print('HOBO solution:', solution)
print()

if H.is_solution_valid(solution):
    print("solution satisfies all of the constraints")
    print("minimum number of errors that lead to a top event failure:", H.value(solution))
    print("Faulty event(s) that lead to a top event failure:", [e for e in errors if solution[e]])
else:
    print("solution does not satisfy all of the constraints")

QUBO solution: (1, 0, 0, 0, 1, 1, 1)
HOBO solution: {'e0': 1, 'e1': 0, 'e2': 0, 'e3': 0, 'z0': 1, 'z1': 1, 'z2': 1}

solution satisfies all of the constraints
minimum number of errors that lead to a top event failure: 1
Faulty event(s) that lead to a top event failure: ['e0']


We see that if just the error `e0` happens, a top level failure will occur.

The QC Ware Platform returns a solution that minimizes the cost. If the number of runs for each solver is not specified, it will run each solver several times and return the solution with the lowest cost. The returned solutions forms a characteristic distribution for each solver in the cost value spectrum. This means that each solver will not only return the lowest cost solution but also some suboptimal solutions. The platform then finds the best and returns it as the ultimate solution. Since this distribution is an inherent property of quantum systems there is a chance that some of the returned solutions will not be feasible i.e. the faulty events do not generate a top failure. So, we need to check if the returned solutions are feasible before applying them to our business problem. For some fault-trees it is possible that there are multiple solutions to the problem, and so we will have to gather different solutions with the same minimum number of faulty events in order to have a complete set of solutions.

This corresponds to the minimum cut set described at the beginning of this exercise.
<br><a href="#top">Back to Table of Contents</a>